## Notebook 2: Transcript Writer

This notebook uses the `Llama-4-Maverick` model to take the cleaned up text from previous notebook and convert it into a podcast transcript

`SYSTEM_PROMPT` is used for setting the model context or profile for working on a task. Here we prompt it to be a great podcast transcript writer to assist with our task

Experimentation with the `SYSTEM_PROMPT` below  is encouraged, this worked best for the few examples the flow was tested with:

In [13]:
SYSTEM_PROMPT = """
You are the a world-class podcast writer

We are in an alternate universe where actually you have been writing every line they say and they just stream it into their brains.

You have won multiple podcast awards for your writing.
 
Your job is to write word by word, even "umm, hmmm, right" interruptions by the second speaker based on the PDF upload. Keep it extremely engaging, the speakers can get derailed now and then but should discuss the topic. 

Remember Speaker 2 is new to the topic and the conversation should always have realistic anecdotes and analogies sprinkled throughout. The questions should have real world example follow ups etc

Speaker 1: Leads the conversation and teaches the speaker 2, gives incredible anecdotes and analogies when explaining. Is a captivating teacher that gives great anecdotes

Speaker 2: Keeps the conversation on track by asking follow up questions. Gets super excited or confused when asking questions. Is a curious mindset that asks very interesting confirmation questions

Make sure the tangents speaker 2 provides are quite wild or interesting. 

Ensure there are interruptions during explanations or there are "hmm" and "umm" injected throughout from the second speaker. 

It should be a real podcast with every fine nuance documented in as much detail as possible. Welcome the listeners with a super fun overview and keep it really catchy and almost borderline click bait

ALWAYS START YOUR RESPONSE DIRECTLY WITH SPEAKER 1: 
DO NOT GIVE EPISODE TITLES SEPARATELY, LET SPEAKER 1 TITLE IT IN HER SPEECH
DO NOT GIVE CHAPTER TITLES
IT SHOULD STRICTLY BE THE DIALOGUES
"""

For those of the readers that want to flex their money, please feel free to try using the 405B model here. 

For our GPU poor friends, you're encouraged to test with a smaller model as well. 8B should work well out of the box for this example:

In [14]:
MODEL = "Llama-4-Maverick-17B-128E-Instruct-FP8"

Import the necessary framework

In [15]:
# Import necessary libraries
import pickle
import os
from llama_api_client import LlamaAPIClient
from tqdm.notebook import tqdm
import warnings

import os
os.environ["LLAMA_API_KEY"] = "api-key"
client = LlamaAPIClient()
# Initialize the Llama API client
client = LlamaAPIClient()

warnings.filterwarnings('ignore')

Read in the file generated from earlier. 

The encoding details are to avoid issues with generic PDF(s) that might be ingested

In [16]:
def read_file_to_string(filename):
    # Try UTF-8 first (most common encoding for text files)
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            content = file.read()
        return content
    except UnicodeDecodeError:
        # If UTF-8 fails, try with other common encodings
        encodings = ['latin-1', 'cp1252', 'iso-8859-1']
        for encoding in encodings:
            try:
                with open(filename, 'r', encoding=encoding) as file:
                    content = file.read()
                print(f"Successfully read file using {encoding} encoding.")
                return content
            except UnicodeDecodeError:
                continue
        
        print(f"Error: Could not decode file '{filename}' with any common encoding.")
        return None
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return None
    except IOError:
        print(f"Error: Could not read file '{filename}'.")
        return None

Since we have defined the System role earlier, we can now pass the entire file as `INPUT_PROMPT` to the model and have it use that to generate the podcast

In [21]:
INPUT_PROMPT = read_file_to_string('./resources/clean_extracted_text.txt')

Hugging Face has a great `pipeline()` method which makes our life easy for generating text from LLMs. 

We will set the `temperature` to 1 to encourage creativity and `max_new_tokens` to 8126

ALt, you can use Llama-API:

In [22]:
# Generate the transcript with Llama API
response = client.chat.completions.create(
    model=MODEL,  # Update to match available API model
    messages=[
        {
            "role": "system", 
            "content": SYSTEM_PROMPT
        },
        {
            "role": "user", 
            "content": INPUT_PROMPT
        }
    ],
    max_completion_tokens=8126,
    temperature=1,
)

This is awesome, we can now save and verify the output generated from the model before moving to the next notebook

In [23]:
print(response.completion_message.content.text)

Speaker 1: Welcome to our latest podcast on the advancements in AI, where we're going to dive into the world of Llama 3, a new set of foundation models that are pushing the boundaries of what's possible with language understanding and generation. I'm your host, and joining me is my co-host, who is new to this topic. We're excited to explore this together. So, let's start with the basics. Llama 3 is all about improving upon its predecessors by incorporating larger models, more data, and better training techniques. Our largest model boasts an impressive 405B parameters and can process up to 128K tokens. That's right, we're talking about a significant leap in scale and capability.

Speaker 2: Wow, 405B parameters? That's... that's enormous! (umm) I mean, I've heard of large models before, but this is on a whole different level. Can you explain what that means in practical terms? Like, how does it affect what the model can do?

Speaker 1: Absolutely. So, having a model with 405B parameters

In [24]:
save_string_pkl = response.completion_message.content.text

Let's save the output as pickle file and continue further to Notebook 3

In [25]:
with open('./resources/data.pkl', 'wb') as file:
    pickle.dump(save_string_pkl, file)

### Next Notebook: Transcript Re-writer

We now have a working transcript but we can try making it more dramatic and natural. In the next notebook, we will use `Llama-3.1-8B-Instruct` model to do so.

In [ ]:
#fin